# the issue is in how the calender is loaded / merged into the rest of the data. The troughs in the final plot should be on xmas day & there shouldn't be such a sudden drop at ~2015. I could try joining in the datetime info earlier? or make one heavily wrangled dataset?

# Imports

In [1]:
import pathlib
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

ROOT = pathlib.Path().absolute().parent
RAW_DATA_PATH = ROOT / 'data' / 'raw'
PROCESSED_DATA_PATH = ROOT / 'data' / 'processed'

# Calendar 

Contains information about the dates the products are sold.
- date: The date in a “y-m-d” format.
- wm_yr_wk: The id of the week the date belongs to.
- weekday: The type of the day (Saturday, Sunday, …, Friday).
- wday: The id of the weekday, starting from Saturday.
- month: The month of the date.
- year: The year of the date.
- event_name_1: If the date includes an event, the name of this event.
- event_type_1: If the date includes an event, the type of this event.
- event_name_2: If the date includes a second event, the name of this event.
- event_type_2: If the date includes a second event, the type of this event.
- snap_CA, snap_TX, and snap_WI: A binary variable (0 or 1) indicating whether the stores of CA,
TX or WI allow SNAP2 purchases on the examined date. 1 indicates that SNAP purchases are
allowed. - SNAP is known colloquially as "Food Stamps"

In [2]:
categorical_variables = [
    'weekday','wday','month',
    'event_name_1','event_type_1',
    'event_name_2','event_type_2']
boolean_variables = ['snap_CA','snap_TX','snap_WI']
dtypes = {var: 'category' for var in categorical_variables}

# read the data with chosen data types
df = pd.read_csv(RAW_DATA_PATH / 'calendar.csv', parse_dates=['date'], dtype=dtypes)
# convert snap columns to boolean
df = df.astype({var: 'boolean' for var in boolean_variables})
# not true - 'd' is used in sales_train_validation.csv
# to match a sales date to a calendar date
# # 'd' is not used - id of the day
# df.drop('d', axis=1, inplace=True)

df.info(memory_usage='deep')
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          1969 non-null   datetime64[ns]
 1   wm_yr_wk      1969 non-null   int64         
 2   weekday       1969 non-null   category      
 3   wday          1969 non-null   category      
 4   month         1969 non-null   category      
 5   year          1969 non-null   int64         
 6   d             1969 non-null   object        
 7   event_name_1  162 non-null    category      
 8   event_type_1  162 non-null    category      
 9   event_name_2  5 non-null      category      
 10  event_type_2  5 non-null      category      
 11  snap_CA       1969 non-null   boolean       
 12  snap_TX       1969 non-null   boolean       
 13  snap_WI       1969 non-null   boolean       
dtypes: boolean(3), category(7), datetime64[ns](1), int64(2), object(1)
memory usage: 197.8 K

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,False,False,False
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,False,False,False
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,False,False,False
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,True,True,False
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,True,False,True


Memory usage reduced approx. 10x


In [3]:
event_names = pd.concat([df.event_name_1,df.event_name_2]).dropna().unique().tolist()
event_types = pd.concat([df.event_type_1,df.event_type_2]).dropna().unique().tolist()

print(f"""There are {len(event_names)} unique events:""")
for row in zip(event_names[:10], event_names[10:20], event_names[20:]):
    print(str('{:<20}' * 3).format(*row))

print(f"""\nThere are {len(event_types)} unique event types:""")
print(*event_types, sep='\n')


There are 30 unique events:
SuperBowl           Mother's day        Halloween           
ValentinesDay       MemorialDay         EidAlAdha           
PresidentsDay       NBAFinalsStart      VeteransDay         
LentStart           NBAFinalsEnd        Thanksgiving        
LentWeek2           Father's day        Christmas           
StPatricksDay       IndependenceDay     Chanukah End        
Purim End           Ramadan starts      NewYear             
OrthodoxEaster      Eid al-Fitr         OrthodoxChristmas   
Pesach End          LaborDay            MartinLutherKingDay 
Cinco De Mayo       ColumbusDay         Easter              

There are 4 unique event types:
Sporting
Cultural
National
Religious


In [4]:
print("Breakdown of event types:")
df.groupby('event_name_1', observed=True).first().groupby('event_type_1').size().sort_values()

Breakdown of event types:


event_type_1
Sporting      3
Cultural      7
National     10
Religious    10
dtype: int64

In [5]:
df[df.event_name_2.notnull()].groupby(
    ['year','month','weekday',
    'event_name_1','event_type_1',
    'event_name_2','event_type_1'], observed=True).size()

year  month  weekday  event_name_1    event_type_1  event_name_2    event_type_1
2011  4      Sunday   OrthodoxEaster  Religious     Easter          Religious       1
2013  5      Sunday   OrthodoxEaster  Religious     Cinco De Mayo   Religious       1
2014  4      Sunday   Easter          Cultural      OrthodoxEaster  Cultural        1
      6      Sunday   NBAFinalsEnd    Sporting      Father's day    Sporting        1
2016  6      Sunday   NBAFinalsEnd    Sporting      Father's day    Sporting        1
dtype: int64

Across the entire 5 year period there are only 5 dates with two events.

In [6]:
display(df.filter(like='snap').sum())

print(
"""Proportion of dates where all states have 'snap' at the same time\n"""
f"""{df.filter(like='snap').all(axis='columns').mean().round(2)}""")

snap_CA    650.0
snap_TX    650.0
snap_WI    650.0
dtype: float64

Proportion of dates where all states have 'snap' at the same time
0.13


Over the 5 year period each state has the same number of 'snap' dates, but they do not fall on the same dates.

## Further Analysis on Calendar

Could look further into the distribution of snap dates throughout the year, how much do the states differ?

What happens when a snap date coincides with an event?

Do the events happen every year & how many occur on the same date? Does it matter that Cinco de Mayo can be any day of the week while LentStart is always Wednesday.


# Sell Prices

Contains information about the price of the products sold per store and date.
- store_id: The id of the store where the product is sold.
- item_id: The id of the product.
- wm_yr_wk: The id of the week.
- sell_price: The price of the product for the given week/store. The price is provided per week
(average across seven days). If not available, this means that the product was not sold during the
examined week. Note that although prices are constant at weekly basis, they may change through
time (both training and test set). 

In [7]:
# need to check that all sell_prices are in whole cents
# load one column of the csv as a series
# find the length of the string right of the decimal point
# (pd
# .read_csv(
#     RAW_DATA_PATH / 'sell_prices.csv',
#     usecols=['sell_price'],
#     squeeze=True)
# .astype(str)
# .str.rsplit('.', n=1)
# .str.len()
# .value_counts())

2    6841121
Name: sell_price, dtype: int64

957.5 MB -> 46.0 MB and far quicker to load

In [8]:
df = pd.read_feather(PROCESSED_DATA_PATH / 'sell_prices_processed.feather')

df.info(memory_usage='deep')
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841121 entries, 0 to 6841120
Data columns (total 7 columns):
 #   Column           Dtype   
---  ------           -----   
 0   state_id         category
 1   store_id         category
 2   cat_id           category
 3   dept_id          category
 4   item_id          category
 5   wm_yr_wk         int16   
 6   sell_price_cent  int16   
dtypes: category(5), int16(2)
memory usage: 65.5 MB


,state_id,store_id,cat_id,dept_id,item_id,wm_yr_wk,sell_price_cent
0,CA,CA_1,HOBBIES,HOBBIES_1,HOBBIES_1_001,11325,958
1,CA,CA_1,HOBBIES,HOBBIES_1,HOBBIES_1_001,11326,958
2,CA,CA_1,HOBBIES,HOBBIES_1,HOBBIES_1_001,11327,826
3,CA,CA_1,HOBBIES,HOBBIES_1,HOBBIES_1_001,11328,826
4,CA,CA_1,HOBBIES,HOBBIES_1,HOBBIES_1_001,11329,826


In [9]:
# import dask.dataframe as dd
# df = dd.read_parquet(PROCESSED_DATA_PATH / 'sell_prices_processed.parquet')
# df.persist().isna().sum().sum().compute()

df.describe(include='all')

,state_id,store_id,cat_id,dept_id,item_id,wm_yr_wk,sell_price_cent
count,6841121,6841121,6841121,6841121,6841121,6.841121e+06,6.841121e+06
unique,3,10,3,7,3049,NaN,NaN
top,CA,TX_2,FOODS,FOODS_3,HOUSEHOLD_1_460,NaN,NaN
freq,2708822,701214,3181789,1835270,2820,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,1.138294e+04,4.410792e+02
std,NaN,NaN,NaN,NaN,NaN,1.486100e+02,3.408868e+02
min,NaN,NaN,NaN,NaN,NaN,1.110100e+04,1.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,1.124700e+04,2.180000e+02
50%,NaN,NaN,NaN,NaN,NaN,1.141100e+04,3.470000e+02
75%,NaN,NaN,NaN,NaN,NaN,1.151700e+04,5.840000e+02


There are no missing values, and as expected we see the 10 stores in 3 states, and 3049 products in 7 departments withoin 3 categories.

In [10]:
print(*df.store_id.cat.categories.to_list(), sep=' | ')

CA_1 | CA_2 | CA_3 | CA_4 | TX_1 | TX_2 | TX_3 | WI_1 | WI_2 | WI_3


So we have 4 stores in CA and 3 each in WI and TX.

In [11]:
df.groupby(['store_id','item_id'], observed=True).first().groupby('store_id').size()

store_id
CA_1    3049
CA_2    3049
CA_3    3049
CA_4    3049
TX_1    3049
TX_2    3049
TX_3    3049
WI_1    3049
WI_2    3049
WI_3    3049
dtype: int64

All stores sell all the products at some time, so different stores haven't chosen to stock different things.

In [12]:
# find the amount of each product type
dep_breakdown = \
(df
.groupby('item_id')
.first()
.groupby(['dept_id'])
.size()
.rename('amount')
.sort_index()
.to_frame()
.reset_index()
.rename(columns={'dept_id':'group'}))

cat_breakdown = \
(df
.groupby('item_id')
.first()
.groupby(['cat_id'])
.size()
.rename('amount')
.sort_index()
.to_frame()
.reset_index()
.rename(columns={'cat_id':'group'}))

breakdown = pd.concat(
    [dep_breakdown,cat_breakdown],
    keys=['departments','categories'],
    names=['supergroup'],
    axis=0).reset_index()

px.bar(breakdown, x='group', y='amount', color='supergroup')

There is some imbalance in product count in categories and departments.

In [13]:
# how many of each category are on sale over time

# first get the number count of each product in each state over time (sum of all stores)
to_plot = \
(df
.groupby(['cat_id','state_id','item_id','wm_yr_wk'], observed=True)
.agg(item_count=('sell_price_cent','count'))
.reset_index())

# divide through by the number of stores in that state
for state, n_stores in {'CA':4,'WI':3,'TX':3}.items():
    to_plot.item_count = np.where(
        to_plot.state_id.values==state, # condition
        to_plot.item_count.values/n_stores, # where true 
        to_plot.item_count.values) # where false

# num of items in that cat in that state in that week   
to_plot = \
(to_plot
.groupby(['cat_id','state_id','wm_yr_wk'], observed=True)
.agg(item_count=('item_count','sum'))
.reset_index())

# match wm_yr_wk to a date
cal_df = pd.read_csv(
    RAW_DATA_PATH / 'calendar.csv',
    parse_dates=['date'],
    usecols=['date','wm_yr_wk'])

# join to dates and resample to find the monthly average
to_plot = \
(to_plot
.merge(
    cal_df,
    left_on=['wm_yr_wk'],
    right_on=['wm_yr_wk'])
.groupby(['cat_id','state_id'])
.resample('M', on='date')
.mean()
.reset_index('date'))

# display(to_plot.sample(5))

category_ids = to_plot.index.get_level_values('cat_id').categories.to_list()
state_ids = to_plot.index.get_level_values('state_id').categories.to_list()

fig = make_subplots(rows=1, cols=3, column_titles=category_ids)

# plot as lines
for col, category_id in enumerate(category_ids):
    for state_id in state_ids:
        tmp = to_plot.loc[(category_id, state_id), :]
        fig.add_trace(
            go.Scattergl(
                x=tmp.date,
                y=tmp.item_count,
                name=state_id,
                legendgroup=state_id,
                showlegend=not col,
                mode='lines',
                marker=go.scattergl.Marker(
                    color={'CA':'blue','TX':'red','WI':'green'}[state_id])),
            row=1, col=col+1)

layout = go.Layout(
    width=1200,
    title="Monthly item count by category and state",
    yaxis_title="Total items available")

fig.update_layout(layout)
fig

Over time more products are added in all 3 categories with the 3 states following very similar paths and slowing down at the start of 2016. The greatest difference is in foods, WI starting with the fewest but adding products the fasts, and CA adding products at the slowest rate.

In [14]:
# # not sure where I'm going with this...
# df_sample = \
# (df
# [['state_id','item_id','wm_yr_wk','sell_price_cent']]
# .sample(frac=0.01))

# (df_sample
# .set_index(['state_id','item_id','wm_yr_wk'])
# .subtract(
#     (df_sample
#     .groupby(['item_id','wm_yr_wk'], observed=True)
#     .agg({'sell_price_cent':'mean'})))
# .query("""sell_price_cent!=0"""))

In [15]:
# mean price of all products in a category over time
to_plot = \
(df
.groupby(['cat_id','state_id','wm_yr_wk'], observed=True)
.agg(mean_item_price=('sell_price_cent','mean'))
.reset_index())

# match wm_yr_wk to a date
cal_df = pd.read_csv(
    RAW_DATA_PATH / 'calendar.csv',
    parse_dates=['date'],
    usecols=['date','wm_yr_wk'])

# resample to find the monthly average
to_plot = \
(to_plot
.merge(
    cal_df,
    left_on=['wm_yr_wk'],
    right_on=['wm_yr_wk'])
.groupby(['cat_id','state_id'])
.resample('M', on='date')
.mean_item_price
.mean()
.reset_index('date'))

# display(to_plot.sample(5))


category_ids = to_plot.index.get_level_values('cat_id').categories.to_list()
state_ids = to_plot.index.get_level_values('state_id').categories.to_list()

fig = make_subplots(rows=1, cols=3, column_titles=category_ids)

# plot as lines
for col, category_id in enumerate(category_ids):
    for state_id in state_ids:
        tmp = to_plot.loc[(category_id, state_id), :]
        fig.add_trace(
            go.Scattergl(
                x=tmp.date,
                y=tmp.mean_item_price,
                name=state_id,
                legendgroup=state_id,
                showlegend=not col,
                mode='lines',
                marker=go.scattergl.Marker(
                    color={'CA':'blue','TX':'red','WI':'green'}[state_id])),
            row=1, col=col+1)

layout = go.Layout(
    width=1200,
    title="Monthly mean item price by category and state",
    yaxis_title="price / cent")

fig.update_layout(layout)
display(fig)
display(fig.for_each_xaxis(lambda x: x.update(range=['2015-01-01','2016-07-01'])))

The average prices over all goods has varied a lot over the last 5 years, but with the introduction of so many new products this is to be expected. If we look just at the last 18 months where very few, if any, new products were added, the prices are far more stable, with household items increasing in price a little across the board and foods costs increasing significantly in Texas and California - possibly due to lack of rainfall.

In [16]:
# are prices of products changing and how?

# mean price of an item in a state across stores
to_plot = \
(df
# .sample(frac=0.01)
.groupby(['cat_id','state_id','item_id','wm_yr_wk'], observed=True)
.agg(item_price=('sell_price_cent','mean'))
.reset_index())

# match wm_yr_wk to a date
cal_df = pd.read_csv(
    RAW_DATA_PATH / 'calendar.csv',
    parse_dates=['date'],
    usecols=['date','wm_yr_wk'])

# join in the dates
to_plot = \
(to_plot
.merge(
    cal_df,
    left_on=['wm_yr_wk'],
    right_on=['wm_yr_wk']))

# standardise the item_price
to_plot.item_price = \
(to_plot
[['state_id','item_id','item_price']]
.groupby(['state_id','item_id'], observed=True)
.transform(lambda x: (x-x.mean())/x.std()))

# get monthly mean item price across states and categories
to_plot = \
(to_plot
.groupby(['cat_id','state_id'], observed=True)
.resample('M', on='date')
.item_price
.mean()
.reset_index('date'))

# display(to_plot.sample(5))

category_ids = to_plot.index.get_level_values('cat_id').categories.to_list()
state_ids = to_plot.index.get_level_values('state_id').categories.to_list()

fig = make_subplots(rows=1, cols=3, column_titles=category_ids)

# plot as lines
for col, category_id in enumerate(category_ids):
    for state_id in state_ids:
        tmp = to_plot.loc[(category_id, state_id), :]
        fig.add_trace(
            go.Scattergl(
                x=tmp.date,
                y=tmp.item_price,
                name=state_id,
                legendgroup=state_id,
                showlegend=not col,
                mode='lines',
                marker=go.scattergl.Marker(
                    color={'CA':'blue','TX':'red','WI':'green'}[state_id])),
            row=1, col=col+1)

layout = go.Layout(
    width=1200,
    title="Monthly (Standardized) mean item price by category and state",
    yaxis_title="standardized price")

fig.update_layout(layout)
fig

All three states follow very similar paths with no one state consistently being more or less expoensive than another. The most obvious feature is the large drop in price in November 2014 which coincides with <a href="https://corporate.walmart.com/newsroom/2014/10/06/walmart-announces-new-commitment-to-a-sustainable-food-system-at-global-milestone-meeting">a commitment to sustainability and affordability of food</a>. The price of food rebopunded a few months latewr but has settled at a price far lower than before nov-2014.

Hobbies products were slowly reducing in price until this date but have siunce risen in price quickly.

In [17]:
# # are items miore expensive in different states?
# state_category_mean_price = df.copy()#.sample(frac=0.01)

# # average price of each product
# overall_item_means = state_category_mean_price.groupby(['category_id','item_id'], observed=True).agg({'sell_price_cent':'mean'})

# # % price difference of each product in each state compared to all states
# price_diff = state_category_mean_price.groupby(['state_id','category_id','item_id'], observed=True).agg({'sell_price_cent':'mean'})
# price_diff = 100 * (price_diff - overall_item_means) / overall_item_means

# # now get the average difference over item categories for each state
# price_diff = \
# (price_diff
# .groupby(['state_id','category_id'])
# .mean()
# .rename(columns={'sell_price_cent':'price_diff'}))

# state_category_mean_price = \
# (state_category_mean_price
# .groupby(['state_id','category_id'], observed=True)
# .agg({'sell_price_cent':'mean'})
# .join(price_diff)
# .reset_index())

# display(state_category_mean_price)

# px.bar(state_category_mean_price, x='category_id', y='price_diff', color='state_id', barmode='group')

In [18]:
# # mean price of all products in a state over time
# to_plot = \
# (df
# # .sample(frac=0.01)
# .groupby(['state_id','wm_yr_wk'], observed=True)
# .agg({'sell_price_cent':'mean'})
# .reset_index())

# # match wm_yr_wk to a date
# cal_df = pd.read_csv(
#     RAW_DATA_PATH / 'calendar.csv',
#     parse_dates=['date'],
#     usecols=['date','wm_yr_wk'])

# # resample to find the monthly average
# to_plot = \
# (to_plot
# .merge(
#     cal_df,
#     left_on=['wm_yr_wk'],
#     right_on=['wm_yr_wk'])
# .groupby('state_id')
# .resample('M', on='date')
# .mean()
# .reset_index('date'))

# # display(to_plot.sample(5))

# # plot as lines
# traces = []
# for state_id in to_plot.index.unique():
#     tmp = to_plot.loc[state_id, :]
#     traces.append(
#         go.Scattergl(
#             x=tmp.date,
#             y=tmp.sell_price_cent,
#             name=state_id,
#             mode='lines'))

# layout = go.Layout(
#     title="Monthly average item price by state",
#     yaxis_title="Price / cent")

# fig = go.Figure(traces, layout)
# fig



## Further Work

- Are prices in different states notably different for the same items
- Are prices different across stores
- are some products seasonal?

# Sales

Contains the historical daily unit sales data per product and store.
- item_id: The id of the product.
- dept_id: The id of the department the product belongs to.
- cat_id: The id of the category the product belongs to.
- store_id: The id of the store where the product is sold.
- state_id: The State where the store is located.
- d_1, d_2, …, d_i, … d_1941: The number of units sold at day i, starting from 2011-01-29.

In [10]:
df = pd.read_feather(PROCESSED_DATA_PATH / 'sales_train_validation_processed.feather')
df.sample(5)

,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
18266,FOODS_3_800,FOODS_3,FOODS,TX_2,TX,5,0,11,5,0,...,3,10,4,8,0,1,4,8,7,7
6654,HOBBIES_2_141,HOBBIES_2,HOBBIES,CA_3,CA,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
25702,HOUSEHOLD_2_215,HOUSEHOLD_2,HOUSEHOLD,WI_2,WI,0,0,0,1,2,...,0,1,0,0,0,0,1,0,0,0
3413,HOBBIES_1_373,HOBBIES_1,HOBBIES,CA_2,CA,0,0,0,0,0,...,3,0,0,1,0,2,1,0,0,0
12320,HOBBIES_1_129,HOBBIES_1,HOBBIES,TX_1,TX,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [11]:
df.iloc[:, :6].info(memory_usage='deep')
print('\n')
df.iloc[:, 6:].info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   item_id   30490 non-null  object  
 1   dept_id   30490 non-null  category
 2   cat_id    30490 non-null  category
 3   store_id  30490 non-null  category
 4   state_id  30490 non-null  category
 5   d_1       30490 non-null  int16   
dtypes: category(4), int16(1), object(1)
memory usage: 2.2 MB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1912 entries, d_2 to d_1913
dtypes: int16(1912)
memory usage: 111.2 MB


Memory reduced 450 MB -> 111 MB

In [127]:
(df
# .drop(columns=['item_id','dept_id','store_id','state_id'])
# .sample(frac=0.01)
)

,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,0,...,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,2,...,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [126]:
pd.read_csv(
    RAW_DATA_PATH / 'calendar.csv',
    # parse_dates=['date'],
    usecols=['d']
    ).d.unique().shape

(1969,)

In [121]:
# match wm_yr_wk to a date
cal_df = pd.read_csv(
    RAW_DATA_PATH / 'calendar.csv',
    parse_dates=['date'],
    usecols=['date','d'])

total_sales = \
(df
.drop(columns=['dept_id','store_id'])
# .sample(frac=0.01)
.rename_axis(columns=['d'])
.melt(
    id_vars=['state_id','cat_id'],
    value_vars=[f'd_{i}' for i in range(1,1914)],
    value_name='sales')
.groupby(['cat_id','state_id','d'])
.agg(total_sales=('sales','sum'))
.reset_index()
.merge(cal_df, left_on='d', right_on='d')
.drop(columns='d'))

# display(to_plot.sample(5))

category_ids = total_sales.cat_id.cat.categories.to_list()
state_ids = total_sales.state_id.cat.categories.to_list()

fig = make_subplots(rows=3, cols=3, column_titles=category_ids, row_titles=['Daily', 'Weekly','Monthly'], shared_xaxes=True)

# plot as lines
for row, resample in enumerate(['D','W','M']):
    # resample data to get weekly / monthly sales
    tmp = \
    (total_sales
    .groupby(['cat_id','state_id'], observed=True)
    .resample(resample, on='date')
    .agg({'total_sales':'sum'})
    .reset_index('date'))

    # for each category and state
    for col, category_id in enumerate(category_ids):
        for state_id in state_ids:

            to_plot = tmp.loc[(category_id, state_id), :]

            fig.add_trace(
                go.Scattergl(
                    x=to_plot.date,
                    y=to_plot.total_sales,
                    name=state_id,
                    legendgroup=state_id,
                    showlegend=(not col) and (not row),
                    mode='lines',
                    marker=go.scattergl.Marker(
                        color={'CA':'blue','TX':'red','WI':'green'}[state_id])),
                row=row+1, col=col+1)

layout = go.Layout(
    width=1200,
    height=600,
    title="Weekly and Monthly Sales")

fig.update_layout(layout)
fig

## To explore

- relative performance of states
- relative performance of stores within states
- seasonal changes in sales
- best performing products
- how are sales on different week days
- sales around holidays